In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
import keras.datasets.mnist as mnist
(train_image, train_label), (test_image, test_label) = mnist.load_data() 

# 数据预处理：多加一个维度

In [3]:
train_image = np.expand_dims(train_image, axis = -1)
test_image = np.expand_dims(test_image, axis = -1)

# 网络搭建

In [7]:
model = keras.Sequential()

In [8]:
from keras import layers

In [9]:
# 第一层要给输入数据的形状：只要管最后3个维度，前面的batch维不用管
model.add( layers.Conv2D( filters=64, kernel_size=(3,3), activation = 'relu', input_shape=(28,28,1) ) )  # 其他一般都用默认
model.add( layers.Conv2D( filters=64, kernel_size=(3,3), activation='relu') )
model.add( layers.MaxPooling2D()  )  # 池化层一般都用默认的

# 进入全连接层：
model.add( layers.Flatten() )  # 把(12,12,64)全部展平为12*12*64 = 9216 —— 前面已经说过这个三维数据里都是特征！！！
model.add(layers.Dense(256, activation='relu'))
model.add( layers.Dropout(0.5) )  # 网络容量还是有些大，dropout一下
model.add(layers.Dense(10, activation='softmax'))  # 最后是10分类输出，激活用softmax多分类

In [10]:
model.compile( optimizer='adam',
               loss = 'sparse_categorical_crossentropy',  # 顺序编码
               metrics=['acc']
)

In [11]:
model.fit(train_image, train_label, epochs = 3, batch_size = 512, validation_data=(test_image, test_label) )


Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 97s 2ms/step - loss: 1.7666 - acc: 0.8795 - val_loss: 0.0685 - val_acc: 0.9778
Epoch 2/3
60000/60000 [==============================] - 94s 2ms/step - loss: 0.0899 - acc: 0.9735 - val_loss: 0.0472 - val_acc: 0.9840
Epoch 3/3
60000/60000 [==============================] - 94s 2ms/step - loss: 0.0648 - acc: 0.9804 - val_loss: 0.0484 - val_acc: 0.9852


# 模型保存

keras的模型保存，有三种形式：
- 保存并加载整个模型（结构 + 权重 + 优化器状态等全部内容都保存）：model.save(起个文件名)  $\quad$ 将Keras模型保存到单个HDF5文件中；
- 只保存并加载模型的结构（层的结构保存，权重等数值参数都丢弃）：json_string = model.to_json()  $\quad$ 没参数
- 只保存模型的权重（以及部分权重）：model.save_weights(起个文件名) $\quad$ 将权重模型保存到单个HDF5文件中；

## 形式1：模型的完整保存（保存、加载、测试）

- 保存：model.save('文件名.h5')
- 加载：从keras.models模块中导入load_model方法，然后：my_model = load_model('模型文件路径')

说明：因为这种保存的模型包含所有内容，因此可以完全直接使用：

In [23]:
# 保存：起个文件名，后缀必须是.h5 —— 保存到当前文件夹下
model.save('my_model.h5')

In [24]:
# 加载：
import keras
from keras.models import load_model

In [25]:
my_model = load_model('E:/Python_code/keras_total/my_model.h5')  # my_model 完全等价于 my_model 完全等价于 model

In [26]:
# 测试载入的模型能不能直接用：
my_model.evaluate(train_image, train_label)

60000/60000 [==============================] - 24s 392us/step


[0.02312193379882762, 0.992816686630249]

In [27]:
# 查看模型的结构：
my_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2359552   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)               

In [28]:
# 测试模型功能：
my_model.predict(test_image[2:3])

array([[8.5051721e-07, 9.9999774e-01, 2.0521943e-08, 2.7576050e-10,
        8.3839876e-07, 2.8289844e-09, 3.0572476e-08, 1.0481748e-07,
        3.3204833e-07, 7.9657326e-08]], dtype=float32)

因为这种方法导入的模型是完整的，因此还可以继续对其“**继续训练**”：

只需再换一批训练数据，输入该模型直接训练即可！该模型还在原有的基础上进一步训练。很方便！

In [29]:
my_model.fit( train_image, train_label, epochs = 1, batch_size=512)

Epoch 1/1
60000/60000 [==============================] - 93s 2ms/step - loss: 0.0498 - acc: 0.9842


## 形式2：只保存模型的层结构（保存、加载、测试）

- 保存：my_model_json = model.to_json()  # model.to_json()的返回值是原模型model的层结构变量，把它写入一个文件即可；
- 加载：从keras.models模块中导入model_from_json方法，然后：model = model_from_json(json_string)

说明：这种方法只保存了模型了层结构，即停留在model.complie()的上一步！—— 也就是说对该层结构可以继续做编译，然后才能训练。

In [31]:
# 保存：
my_model_json1 = model.to_json()

# 把my_model_json要写入一个文件中：当前文件夹下会多一个“my_model_json.json”的文件，它保存了层结构
with open('my_model_json.json', 'w') as f:
    f.write(my_model_json1)

In [33]:
# 加载：
from keras.models import model_from_json

# 先把那个my_model_json.json的文件读进来，并用my_model_json2变量绑定：
with open('my_model_json.json', 'r') as f:
    my_model_json2 = f.read()
    
# 开始加载：
model = model_from_json(my_model_json2)

In [34]:
# 查看模型结构：

In [35]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2359552   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)               

In [36]:
# 查看模型是否具有功能：不能，因为当前模型是停留在model.compile()之前那里。
model.evaluate(train_image, train_label)

RuntimeError: You must compile a model before training/testing. Use `model.compile(optimizer, loss)`.

In [37]:
# 对该模型进行编译，查看其功能：
model.compile( optimizer='adam',
               loss = 'sparse_categorical_crossentropy',  # 顺序编码
               metrics=['acc']
)

In [38]:
model.evaluate(train_image, train_label)

60000/60000 [==============================] - 24s 396us/step


[13.230497810618083, 0.12568333745002747]

结果很差，说明当前该模型的权重都是最开始“完全随机”的状态，说明：确实只保存了“层结构”，没有保存之前训练后的各种权重参数！

也就是说明这种保存的模型，完完全全就是model.compile()之前的那个状态。

## 形式3：只保存模型的权重（保存、加载、测试） 

- 保存：model.save_weights('起个文件名.h5')
- 加载：从keras.models模块中导入load_model方法，然后：model.load_weights('模型文件路径')

说明：这种权重的加载，必须是在“已有层结构的模型”下加载，即model.load_weights中的model“至少”已把层结构搭建好了。—— 好理解：如果连层都没搭建，那这些依赖不同层的权重赋值给谁？

当然：这种已经搭建好层结构的model，既可以是自己创建的，当然也可以是用“形式2”载入的“层结构模型”。

为了更好的展示，换到文件24中专门展示这种“形式3”的“模型参数保存”的用法。